# How to use OpenAI's API for inference 

<p>This notebook demonstrates a script to use OpenAI's API and ask GPT-4 to perform a custom entity extraction task </p>

In [ ]:
from openai import OpenAI
import pandas as pd

In [ ]:
# insert your openAI API key here
client = OpenAI(
api_key="your API key",
)

<p>The following cell defines a function to make inference with a selected model from OpenAI.
In this case, GPT-4 Turbo is selected. <b>Temperature</b> parameter is set to 0, to reduce the randomness in the model's generated text. The prompt is inside <b>messages</b>: it is composed of a series of instructions and three examples (POST and ANSWER).The function takes a post as input, passes it to the model after the prompt, and returns the content of the message generated by the LLM.</p> 

In [5]:
def query(post):
    response = client.chat.completions.create(
        # model selection
        model= "gpt-4-turbo",
        # temperature parameter to control randomness
        temperature=0,
        # messages: a list of messages used to construct the dialogue or prompt for the model
        messages= [
            # first message: prompt to describe the task
        {"role":"user", "content":"""You are a customer service expert. I am going to provide you a TripAdvisor
        post about travels, you should identify the needs that the user expressed in the text.
        Needs should be expressed as short sentences. The answer should be generated as a list of needs separated by a comma.
        Avoid any unnecessary text or comment.
        Here are some examples:
        POST: My husband and I will be visiting NYC for the first time late April and plan to spend some time in Central Park. We enjoy just walking through parks, but I was wondering if there are any specific attractions we should make sure we see while in the park? Also is there a nice place to eat lunch with a view of the park? thank you!!!
        ANSWER: Suggestions on must-see attraction near Central Park, Restaurant recommendation.
        POST: I've come across a Cyber Sale announced by the Sonesta group (used to be Affinia) for their three hotels in NYC. The sale goes through April 2024 and looks like the rates are refundable but you must stay at least 2 nights. Cheapest dates are January and February as usual but over the Thanksgiving week / few days it's not too bad either. https://www.affinia.com/special-offers/affinia-cyber-sale Any other good offers at other hotels are welcome to be posted in this thread as well.
        ANSWER: Affordable accommodations, Accomodations advices
        POST: Hi We are a family of 4, 2 adults, 2 daughters (12yrs & 14yrs). We are flying London to NY (to and from JFK) on Thurs 24th Oct, landing at 3pm, then leaving Sat 26th Oct at midday. It's an extended stop over on route to our main holiday 😊 We're staying at Sohotel, Bowrey & I wanted to ask advice on travel/ itinerary. We've found the forums so helpful as the advice has meant we've narrowed down things we'd like to do in the short time we have! Getting to the hotel: At the time of day we land, and with one suitcase/small backpacks, is it better to get the air train/subway or a taxi? Itinerary: Thursday eve, we thought we'd keep it simple and low stress, so Times Square and dinner. Friday we've narrowed it simply to Brooklyn Bridge, Ghostbusters firehouse (we're 80s kids, and our girls love the films!), the free ferry to Statue of Liberty...maybe central park. Can we squeeze in TOTR in the evening? We need to work out which order to do things 😬 Saturday is pack and go - again is the public transport ok in the morning for the flight, or book a taxi? Many thanks for any guidance, I hope I've given enough info!
        ANSWER: Transportation Advice, Itinerary Planning, Efficiency Concerns, Departure Transportation
        """},
            # second message: answer from the model
        {"role":"assistant", "content":"Ok, I'm ready."},
            # third message: the input text
        {"role":"user", "content":post},
        ]
    )
    return response.choices[0].message.content


<p>You can test the function by calling it with a post as input. It prints the extracted customer needs.</p>

In [2]:
print(query("""Hi all We are 3 adults who have been to NYC multiple times. 
          We'll be in NYC Dec 3-8 and the hotels in Manhattan are off the charts . 
          Looking for a hotel room with 2 beds outside of Manhattan for $250- 300 per night.
          I know it might be impossible but I figured if anyone could help us y'all could.
          I posted in the Brooklyn forum and it was recommended that I post here too. Thanks Geri 4"""))

Affordable hotel recommendations outside Manhattan, Budget accommodation options


<p>Now we can apply it to all the posts of the TripAdvisor dataset</p>

In [4]:
# import the dataset
df = pd.read_excel(r"..\data\tripAdvisor_posts.xlsx")
df.head(5)

,TEXT,label
0,Hey fellow travel enthusiasts! 👋 So here's ...,suggestions upon best christmas activities and...
1,Hello! We will be taking a family trip (teens ...,accommodation advices
2,Where could I deposit luggage nearby Penn Stat...,luggage storage options
3,Hi i have posted a few times before and really...,"itinerary planning advices, shopping advices, ..."
4,Hi looking for advice on where to stay on a bu...,"accommodation advices, travelling advices from..."


In [21]:
# For each post of the dataset, call the function and save the output
output = []
for post in df["TEXT"]:
    answer = query(post)
    output.append(answer)

--- 1265.0873124599457 seconds ---


In [22]:
df["extracted_needs"] = output

In [ ]:
# Save the new dataset with the extracted needs
df.to_excel(r"..\data\gpt4_output.xlsx")

<p>How can we evaluate whether the model's needs extraction is correct or good?</p>